In [25]:
import dxlib as dx
import pandas as pd
from dxlib.interfaces.external.yfinance import YFinanceAPI

In [11]:
tickers = ["AAPL", "MSFT", "GOOGL"]
security_manager = dx.SecurityManager.from_list(tickers)

In [22]:
schema = dx.Schema(
    fields=["open", "high", "low", "close", "volume", "adj_close"],
    levels=[dx.SchemaLevel.DATE, dx.SchemaLevel.SECURITY],
    security_manager=security_manager
)
my_history = dx.History(schema=schema)

In [29]:
start = dx.Date.prevdays(365)
end = dx.Date.today()
api = YFinanceAPI()
data = api.historical(tickers, start, end)

In [83]:
class MyStrategy(dx.Strategy):
    def execute(
            self, observation: any, history: dx.History, position: dx.Inventory
    ) -> pd.Series:
        # No nosso caso, da todos os valores de datas unicas e acoes unicas
        levels = history.levels_unique()  
        # Le todos valores unicos no nivel

        idx, _ = observation

        signals = pd.Series(
            dx.Signal(dx.Side.WAIT), index=pd.MultiIndex.from_tuples([idx], names=levels.keys())
        )
        
        security_level = history.schema.levels.index(dx.SchemaLevel.SECURITY)
        security = idx[security_level]
        
        if security.ticker == "AAPL":
            signals.loc[idx] = dx.Signal(dx.Side.BUY, 1)
        elif security.ticker == "MSFT":
            signals.loc[idx] = dx.Signal(dx.Side.SELL, 1)
        elif security.ticker == "GOOGL":
            signals.loc[idx] = dx.Signal(dx.Side.WAIT, 1)

        return signals

In [84]:
rsi_strategy = dx.strategies.RsiStrategy()

In [85]:
strategy = MyStrategy()
executor = dx.Executor(strategy)

In [86]:
signals: dx.History = executor.run(data)

In [87]:
signals

                                            signal
date                security                      
2023-05-16 10:30:00 AAPL (equity)    BUY: 1 @ None
2023-05-17 10:30:00 AAPL (equity)    BUY: 1 @ None
2023-05-18 10:30:00 AAPL (equity)    BUY: 1 @ None
2023-05-19 10:30:00 AAPL (equity)    BUY: 1 @ None
2023-05-22 10:30:00 AAPL (equity)    BUY: 1 @ None
...                                            ...
2024-05-09 10:30:00 GOOGL (equity)  WAIT: 1 @ None
2024-05-10 10:30:00 GOOGL (equity)  WAIT: 1 @ None
2024-05-13 10:30:00 GOOGL (equity)  WAIT: 1 @ None
2024-05-14 10:30:00 GOOGL (equity)  WAIT: 1 @ None
2024-05-15 16:33:49 GOOGL (equity)  WAIT: 1 @ None

[756 rows x 1 columns]

In [88]:
aapl = data.schema.security_manager.get("MSFT")
signals.get(levels={dx.SchemaLevel.SECURITY: [aapl]})

                                           signal
date                security                     
2023-05-16 10:30:00 MSFT (equity)  SELL: 1 @ None
2023-05-17 10:30:00 MSFT (equity)  SELL: 1 @ None
2023-05-18 10:30:00 MSFT (equity)  SELL: 1 @ None
2023-05-19 10:30:00 MSFT (equity)  SELL: 1 @ None
2023-05-22 10:30:00 MSFT (equity)  SELL: 1 @ None
...                                           ...
2024-05-09 10:30:00 MSFT (equity)  SELL: 1 @ None
2024-05-10 10:30:00 MSFT (equity)  SELL: 1 @ None
2024-05-13 10:30:00 MSFT (equity)  SELL: 1 @ None
2024-05-14 10:30:00 MSFT (equity)  SELL: 1 @ None
2024-05-15 16:33:47 MSFT (equity)  SELL: 1 @ None

[252 rows x 1 columns]